In [1]:
# Import the Earth Engine Python Package
import ee

# Initialize the Earth Engine object, using the authentication credentials.
ee.Initialize()

In [2]:
# Import the required Images and ImageCollections
sn = ee.FeatureCollection("ft:1VX1Uny1uAIYdgfuvsjLJVw1wMkhmSZq7fCQ0gowV");
modis = ee.ImageCollection("MODIS/006/MOD13Q1");
conifer_forest = ee.Image("users/mkoontz/sierra-nevada-250m-calveg-conifer-forested-pixels-by-whr-type-no-mask");

# How many images are in the MODIS ImageCollection?
print(modis.size().getInfo());

391


In [25]:
# Display an image to make sure everything is working as expected.

from IPython.display import Image, display, HTML

thumburl = conifer_forest.getThumbUrl({
                'min':0, 
                'max':1          
            })

#print thumburl
img_thumb = Image(url=thumburl)
display(img_thumb)

In [5]:
# Define the masking function that will determine which pixels are both forested AND of high MODIS quality.

def quality_forest_mask(img):
  the_mask = (
    img
        .select(["SummaryQA"]) # Look at the SummaryQA band
        .eq(0)                 # Return 1 if value is 0 (good quality)
        .And(conifer_forest)  # ... AND if pixel is forested. 
  );

  return img.updateMask(the_mask);         # Mask pixels with a value of 0

In [15]:
# Map the masking function to the MODIS image collection

m_modis = modis.map(quality_forest_mask);
print(ee.Image(m_modis.first()).getInfo());

{'version': 1485050354780000, 'type': 'Image', 'id': 'MODIS/006/MOD13Q1/2000_02_18', 'properties': {'system:time_end': 952214400000, 'system:asset_size': 28219636986, 'system:footprint': {'type': 'LinearRing', 'coordinates': [[-180.0, 90.0], [-180.0, -90.0], [180.0, -90.0], [180.0, 90.0], [-180.0, 90.0]]}, 'system:index': '2000_02_18', 'system:time_start': 950832000000}, 'bands': [{'data_type': {'min': -32768, 'type': 'PixelType', 'precision': 'int', 'max': 32767}, 'crs': 'SR-ORG:6974', 'dimensions': [4657, 2530], 'origin': [38675, 23795], 'id': 'NDVI', 'crs_transform': [231.65635826395825, 0.0, -20015109.353988, 0.0, -231.65635826395834, 10007554.676994]}, {'data_type': {'min': -32768, 'type': 'PixelType', 'precision': 'int', 'max': 32767}, 'crs': 'SR-ORG:6974', 'dimensions': [4657, 2530], 'origin': [38675, 23795], 'id': 'EVI', 'crs_transform': [231.65635826395825, 0.0, -20015109.353988, 0.0, -231.65635826395834, 10007554.676994]}, {'data_type': {'min': 0, 'type': 'PixelType', 'precis

In [29]:
# Test the first image
from IPython.display import Image, display, HTML
test_img = ee.Image(m_modis.first()).select(["EVI"]).clip(sn);
test_img = ee.Image(modis.first()).select(["EVI"]).clip(sn);

thumburl = test_img.getThumbUrl({
                'min':0, 
                'max':100,
            })

#print thumburl
img_thumb = Image(url=thumburl)
display(img_thumb)

AttributeError: 'Element' object has no attribute 'select'